# This is a notebook for querying the merged Eutro WB for Chl, Oxygen and Nutrients instance for the 1.0.0 Beacon release.

- You can run each cell individually by pressing "shift + enter".
- For more information, questions, bugs, please contact us on Slack:
  - https://join.slack.com/t/beacontechnic-wwa5548/shared_invite/zt-2dp1vv56r-tj_KFac0sAKNuAgUKPPDRg.


#### In order to get access to the Beacon endpoint, you need to fill in your unique personal token between the " " in the cell below.


In [ ]:
Token = ""

#### Install the following packages, if you have not already installed them in your environment:

- pip install requests
- pip install xarray
- pip install ipywidgets
- pip install cartopy
- pip install h5netcdf
- pip install netcdf4
- pip install scipy
- pip install packaging


#### Import the required packages


In [3]:
import requests
import json
import xarray as xr
import datetime
import pandas as pd
import os
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt

#### Retrieve the available columns from the Merged TS endpoint

Swagger page here: https://beacon-wb1-ts.maris.nl/swagger/


In [ ]:
responseinfo = requests.get("https://beacon-wb2-eutrophication.maris.nl/api/query/available-columns",
                            headers={"Authorization": f"Bearer {Token}"})
params = responseinfo.json()

#### Below you can search through the available columns by entering text between the brackets of search_columns(" ").


In [ ]:
def search_columns(search_term):
    search_term = search_term.lower()
    matches = [col for col in params if search_term in col.lower()]

    if matches:
        print("Matching columns:")
        for match in matches:
            print(match)
    else:
        print("No matching columns found.")


search_columns("common")  # Enter your search term here

Matching columns:
Common.Depth
Common.EDMO_CODE
Common.Latitude
Common.Longitude
Common.Oxygen
Common.Oxygen.P01
Common.Oxygen.P06
Common.Oxygen.qf
Common.Salinity
Common.Salinity.P01
Common.Salinity.P06
Common.Salinity.qf
Common.Temperature
Common.Temperature.P01
Common.Temperature.P06
Common.Temperature.qf
Common.Time
Common.source.beacon


#### You can define here your input parameters


In [ ]:
parameter = "Common.Oxygen"  # column name
mindate = "2015-01-01"  # yyyy-mm-dd
maxdate = "2015-06-30"  # yyyy-mm-dd
minlon = -180
maxlon = 180
minlat = -90
maxlat = 90
mindepth = 0
maxdepth = 10

#### This will create the query body based on your input parameters, you can add other "query_parameters" and "filters" to suit your needs.

- For more query examples and explanations, you can take a look at https://github.com/maris-development/beacon-blue-cloud.


In [ ]:
def query(parameter, mindate, maxdate, minlon, maxlon, minlat, maxlat, mindepth, maxdepth):
    body = {
        "query_parameters": [
            {
                "column_name": parameter,
                "alias": parameter,
                "skip_fill_values": True
            },
            {
                "column_name": f"{parameter}.P06",
                "alias": "Unit"
            },
            {
                "column_name": "Common.Time",
                "alias": "datetime"
            },
            {
                "column_name": f"{parameter}.qf",
                "alias": f"{parameter}_qf"
            },
            {
                "column_name": f"{parameter}.P01",
                "alias": f"{parameter}_P01"
            },
            # {
            #     "column_name": f"{parameter}.P06",
            #     "alias": f"{parameter}_P06"
            # },
            {
                "column_name": "Common.Depth",
                "alias": "Depth"
            },
            {
                "column_name": "Common.Latitude",
                "alias": "LONGITUDE"
            },
            {
                "column_name": "Common.Longitude",
                "alias": "LATITUDE"
            },
            {
                "column_name": "Common.source.beacon",
                "alias": "Collection"
            }
        ],
        "filters": [
            {
                "for_query_parameter": "datetime",
                "min": f"{mindate}T00:00:00",
                "max": f"{maxdate}T00:00:00",
                "cast": "timestamp"
            },
            {
                "for_query_parameter": "Depth",
                "min": mindepth,
                "max": maxdepth
            },
            {
                "for_query_parameter": "LONGITUDE",
                "min": minlon,
                "max": maxlon
            },
            {
                "for_query_parameter": "LATITUDE",
                "min": minlat,
                "max": maxlat
            }
        ],
        "output": {
            "format": "netcdf"
        }}
    return body


query_body = query(parameter, mindate, maxdate, minlon,
                   maxlon, minlat, maxlat, mindepth, maxdepth)

#### This is the post request that is sent to Beacon with the above specified body.


In [ ]:
response = requests.post("https://beacon-wb2-eutrophication.maris.nl/api/query", json.dumps(query_body), headers={
    "Authorization": f"Bearer {Token}",
    "Content-type": "application/json"
})

if response.status_code == 204:
    print("No data has been found for your query, please update your input fields above and run the notebook again.")
elif response.status_code != 200:
    print(response.text)

In [ ]:
regionname = f"[{minlat},{minlon}],[{maxlat},{maxlon}]"

if not os.path.exists("./Beacon_V1.0.0_Output"):
    os.makedirs("Beacon_V1.0.0_Output")

open(
    f"./Beacon_V1.0.0_Output/EWB_merged_{parameter}_{regionname}_{mindate}-{maxdate}_[{mindepth}-{maxdepth}m].nc", "wb").write(response.content)
df = xr.open_dataset(
    f"./Beacon_V1.0.0_Output/EWB_merged_{parameter}_{regionname}_{mindate}-{maxdate}_[{mindepth}-{maxdepth}m].nc").to_dataframe()
df = df.assign(datetime=pd.to_datetime(
    df["datetime"])).set_index("datetime").sort_index()
df

,Common.Oxygen,Unit,Common.Oxygen_qf,Common.Oxygen_P01,Depth,LONGITUDE,LATITUDE,Collection,dataset_id
datetime,,,,,,,,,
2015-01-01 12:32:00,228.650589,KGUM,1,DOXMZZXX,0.595873,30.910919,-26.975128,EMODnet Chemistry,8.0
2015-01-01 12:32:00,228.597916,KGUM,1,DOXMZZXX,1.390367,30.910919,-26.975128,EMODnet Chemistry,8.0
2015-01-01 12:32:00,228.633072,KGUM,1,DOXMZZXX,9.930987,30.910919,-26.975128,EMODnet Chemistry,8.0
2015-01-01 12:32:00,228.687485,KGUM,1,DOXMZZXX,9.037218,30.910919,-26.975128,EMODnet Chemistry,8.0
2015-01-01 12:32:00,228.629486,KGUM,1,DOXMZZXX,8.143445,30.910919,-26.975128,EMODnet Chemistry,8.0
...,...,...,...,...,...,...,...,...,...
2015-06-29 22:01:00,236.441833,KGUM,1,DOXMZZXX,5.955148,37.770000,-31.825989,EMODnet Chemistry,73.0
2015-06-29 22:01:00,236.433380,KGUM,1,DOXMZZXX,4.962635,37.770000,-31.825989,EMODnet Chemistry,73.0
2015-06-29 22:01:00,236.453720,KGUM,1,DOXMZZXX,0.000000,37.770000,-31.825989,EMODnet Chemistry,73.0


#### Optionally apply a filter on your parameter for quick removal of outliers. Note that this can also be achieved within your Beacon request by applying a filter on your parameter.


In [ ]:
highbound = 40
lowbound = -2
df = df.loc[(df[f"{parameter}"] < highbound) & (df[f"{parameter}"] > lowbound)]
df

#### Plotting of results.


In [ ]:
fig = plt.figure(figsize=(23, 18))
ax = plt.axes(projection=ccrs.PlateCarree())

ax.set_extent([-180, 180, -90, 90], crs=ccrs.PlateCarree())  # (west, east, south, north)

ax.coastlines(resolution="10m")
ax.gridlines(draw_labels=True)

sc = ax.scatter(df["LONGITUDE"], df["LATITUDE"], c=df[parameter], cmap="viridis", s=15, transform=ccrs.PlateCarree())

ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.OCEAN)
ax.add_feature(cfeature.RIVERS)     
ax.add_feature(cfeature.BORDERS)   
ax.add_feature(cfeature.LAKES, alpha=0.1) 

cbar = plt.colorbar(sc, ax=ax, orientation="vertical", shrink=0.6, label="Value Set 1")
cbar.set_label(f"{parameter} [{df["Unit"].iloc[0]}]")

plt.title(f"{parameter} {regionname} {mindate}-{maxdate} [{mindepth}-{maxdepth}m]")

plt.show()